In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("RF_datasets.csv")

In [3]:
df.describe()

,age,salary,experience_years,work_hours_per_week,performance_score,promotion_last_5years,target_left_company
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000
mean,40.986000,85084.980000,19.34500,49.554000,2.970316,0.511000,0.497000
std,13.497852,37146.053131,11.45492,11.255833,1.128795,0.500129,0.500241
min,18.000000,20060.000000,0.00000,30.000000,1.000000,0.000000,0.000000
25%,29.000000,53704.250000,10.00000,40.000000,2.000000,0.000000,0.000000
50%,42.000000,84772.000000,19.00000,49.000000,2.970316,1.000000,0.000000
75%,52.000000,116535.500000,29.00000,59.000000,3.900000,1.000000,1.000000
max,64.000000,149972.000000,39.00000,69.000000,5.000000,1.000000,1.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   age                    1000 non-null   int64  
 1   salary                 1000 non-null   float64
 2   experience_years       1000 non-null   int64  
 3   education_level        1000 non-null   object 
 4   department             1000 non-null   object 
 5   city_tier              1000 non-null   object 
 6   work_hours_per_week    1000 non-null   int64  
 7   performance_score      1000 non-null   float64
 8   promotion_last_5years  1000 non-null   int64  
 9   target_left_company    1000 non-null   int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 78.3+ KB


In [8]:
# check for missing values
df.isnull().sum()

age                      0
salary                   0
experience_years         0
education_level          0
department               0
city_tier                0
work_hours_per_week      0
performance_score        0
promotion_last_5years    0
target_left_company      0
dtype: int64

In [9]:
df['target_left_company'].value_counts()

target_left_company
0    503
1    497
Name: count, dtype: int64

In [11]:
# Conver categorical columns to numerical using one-hot encoding
from sklearn.preprocessing import OneHotEncoder     

ohe = OneHotEncoder(sparse_output=False, drop='first')

cat_cols = ['city_tier', 'department', 'education_level']

ohe_df = pd.DataFrame(ohe.fit_transform(df[cat_cols]), columns=ohe.get_feature_names_out(cat_cols))

df = pd.concat([df.drop(columns=cat_cols), ohe_df], axis=1)

df.head()

,age,salary,experience_years,work_hours_per_week,performance_score,promotion_last_5years,target_left_company,city_tier_Tier-2,city_tier_Tier-3,department_HR,department_IT,department_Marketing,department_Operations,education_level_High School,education_level_Master,education_level_PhD
0,56,136748.0,33,47,3.800000,0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,46,25287.0,28,40,2.400000,1,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,32,146593.0,3,45,2.970316,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,60,54387.0,16,47,2.600000,0,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,25,28512.0,34,64,1.900000,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
X = df.drop(columns=['target_left_company'])
X.head()

,age,salary,experience_years,work_hours_per_week,performance_score,promotion_last_5years,city_tier_Tier-2,city_tier_Tier-3,department_HR,department_IT,department_Marketing,department_Operations,education_level_High School,education_level_Master,education_level_PhD
0,56,136748.0,33,47,3.800000,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,46,25287.0,28,40,2.400000,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,32,146593.0,3,45,2.970316,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,60,54387.0,16,47,2.600000,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,25,28512.0,34,64,1.900000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
y = df['target_left_company']

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100,random_state=42)

model = rf_model.fit(X_train, y_train)

y_pred = model.predict(X_test)

y_pred

array([1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0])

In [26]:
# Model Evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred))

Confusion Matrix:
 [[54 36]
 [64 46]]

Classification Report:
               precision    recall  f1-score   support

           0       0.46      0.60      0.52        90
           1       0.56      0.42      0.48       110

    accuracy                           0.50       200
   macro avg       0.51      0.51      0.50       200
weighted avg       0.51      0.50      0.50       200


Accuracy Score: 0.5
